In [1]:
library(dplyr)
library(stringr)
library(readr)
library(tidyr)
library(lfe)
library(scales)
library(repr)
library(stargazer)
library(gridExtra)
options(repr.plot.width=10, repr.plot.height=10)
source('~/covid_mobility/paper_code/utils/adjacency_and_cluster_robust_inference.R')
source('~/covid_mobility/paper_code/utils/residualizer+genAlters.R')
source('../utils/labels_and_colors.r')
load('/nfs/sloanlab004/projects/covid_mobility_proj/data/PROCESSED_DATA/social_adj_matrix.RData')
load('/nfs/sloanlab004/projects/covid_mobility_proj/data/PROCESSED_DATA/data_v2.RData')


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack



Attaching package: ‘scales’


The following object is masked from ‘package:readr’:

    col_factor



Please cite as: 


 Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.2. https://CRAN.R-project.org/package=stargazer 



Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine



Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching packa

In [ ]:
options(repr.plot.width=5, repr.plot.height=10)
ggplot(panel, aes(x = PRCP, y = sg_nhd, color = as.factor(stay_home))) + 
    geom_point(alpha = .2, size = .25) + 
    coord_cartesian(xlim = c(0, 80)) +
    geom_smooth(method = 'lm', se = F) +
    theme(legend.position = 'bottom')

In [ ]:
options(repr.plot.width=5, repr.plot.height=10)
ggplot(panel, aes(x = TMAX, y = sg_nhd, color = as.factor(stay_home))) + 
    geom_point(alpha = .2, size = .25) + 
    geom_smooth(method = 'lm', formula = 'y ~ x + I(x^2)', se = F) +
    coord_cartesian(xlim= c(-5, 35)) + 
    theme(legend.position = 'bottom')

In [ ]:
panel %>%
    select(county_fips, ds, sg_nhd, sg_mcbgv, fb_rnstu, fb_btvrc) %>%
    gather(key = 'DV', value = 'Y', -county_fips, -ds) %>%
    left_join(panel %>% select(county_fips, ds, PRCP, stay_home)) %>%
    ggplot(aes(x = PRCP, y = Y, color = as.factor(stay_home))) + 
    geom_point(alpha = .2, size = .25) + 
    coord_cartesian(xlim = c(0, 80)) +
    geom_smooth(method = 'lm', se = F) +
    facet_grid(. ~ DV, scales = 'free') +
    theme(legend.position = 'bottom') 

In [ ]:
panel %>%
    mutate(prcp.)

prcp_res <- felm(PRCP ~ stay_home + ban_gatherings + ban_gmr | 0 | 0 | 0, data=panel,
                 weights=panel$n)
nhd_res  <- felm(sg_nhd ~ stay_home + ban_gatherings + ban_gmr | 0 | 0 | 0, data=panel,
                 weights=panel$n)
temp_res <- felm(TMAX ~ stay_home + ban_gatherings + ban_gmr | 0 | 0 | 0, data=panel,
                 weights=panel$n)

panel$prcp_res <- as.vector(residuals(prcp_res))
panel$nhd_res <- as.vector(residuals(nhd_res))
panel$temp_res <- as.vector(residuals(temp_res))
panel <- cbind(panel, weightedAlters.mod(panel, WM, 'alter_prcp_res', prcp_res))
panel <- cbind(panel, weightedAlters.mod(panel, WM, 'alter_nhd_res', nhd_res))
panel <- cbind(panel, weightedAlters.mod(panel, WM, 'alter_temp_res', temp_res))

In [ ]:
rain_instr_plot <- panel %>%
  ggplot(., aes(x=alter_prcp_res, y=alter_nhd_res)) + geom_point(alpha=.05) + geom_smooth(method='lm', se=FALSE) + 
  xlab('Average residual alter rainfall') + ylab('Average residual alter\ndevices not at home') + 
  stat_summary_bin(fun='mean', bins=50, size=2, geom='point', color='red') + 
  xlim(c(min(panel$alter_prcp_res), quantile(panel$alter_prcp_res, .95))) + 
  scale_color_manual(values = outcome_source_colors)

In [ ]:
ggsave(
  paste0("../output/rain_instr_plot.pdf"),
  plot = rain_instr_plot,
  device = NULL,
  path = NULL,
  scale = 1.2,
  width = 7.5,
  height = 6,
  units = c("in"),
  dpi = 200,
  limitsize = FALSE,
  useDingbats=FALSE
)

In [ ]:
tmp_instr_plot <- panel %>%
  ggplot(., aes(x=alter_temp_res, y=alter_nhd_res)) + geom_point(alpha=.05) + geom_smooth(method='lm', se=FALSE) + 
  xlab('Average residual alter temperature') + ylab('Average residual alter\ndevices not at home') + 
  stat_summary_bin(fun='mean', bins=50, size=2, geom='point', color='red') + 
  xlim(c(min(panel$alter_temp_res), quantile(panel$alter_temp_res, .95))) + 
  scale_color_manual(values = outcome_source_colors)

In [ ]:
ggsave(
  paste0("../output/tmp_instr_plot.pdf"),
  plot = tmp_instr_plot,
  device = NULL,
  path = NULL,
  scale = 1.2,
  width = 7.5,
  height = 6,
  units = c("in"),
  dpi = 200,
  limitsize = FALSE,
  useDingbats=FALSE
)